In [17]:
# installing nltk
!pip install nltk
# Downloading a book 'Nineteen hundred?: A forecast and a story by Marianne Farningham' from www.gutenberg.org
! wget https://www.gutenberg.org/files/69638/69638-0.txt

--2022-12-28 12:20:14--  https://www.gutenberg.org/files/69638/69638-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 832244 (813K) [text/plain]
Saving to: ‘69638-0.txt’

69638-0.txt         100%[===================>] 812.74K   526KB/s    in 1.5s    

2022-12-28 12:20:17 (526 KB/s) - ‘69638-0.txt’ saved [832244/832244]



## 1. Load and preprocess data
* Load and tokenize data.
* Split the sentences into train and test sets.
* Replace words with a low frequency by an unknown marker <unk>

In [18]:
import nltk
import random

# Load and tokenize data
nltk.download('punkt')
with open('69638-0.txt', 'r') as f:
    data = f.read()

# Split the sentences into train and test sets
sentences = nltk.sent_tokenize(data)
random.shuffle(sentences)
train_size = int(len(sentences) * 0.8)
train_sentences = sentences[:train_size]
test_sentences = sentences[train_size:]

# Replace words with a low frequency by an unknown marker <unk>
word_freq = nltk.FreqDist(nltk.word_tokenize(data))
unk_threshold = 3
train_sentences = [[word if word_freq[word] > unk_threshold else '<unk>' for word in nltk.word_tokenize(sentence)] for sentence in train_sentences]
test_sentences = [[word if word_freq[word] > unk_threshold else '<unk>' for word in nltk.word_tokenize(sentence)] for sentence in test_sentences]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 2. Develop N-gram based language models
* Compute the count of n-grams from a given data set.
* Estimate the conditional probability of a next word with k-smoothing.

In [23]:
from collections import defaultdict

def compute_ngram_counts(sentences, n):
    # Compute the count of n-grams from a given data set
    ngram_counts = defaultdict(int)
    for sentence in sentences:
        for i in range(len(sentence) - n + 1):
            ngram = tuple(sentence[i:i+n])
            ngram_counts[ngram] += 1
    return ngram_counts

def estimate_conditional_prob(ngram, ngram_counts, k, vocab_size):
    # Estimate the conditional probability of a next word with k-smoothing
    numerator = ngram_counts[ngram] + k
    denominator = ngram_counts[ngram[:-1]] + k * vocab_size
    return numerator / denominator

# Compute the count of unigrams, bigrams, and trigrams
unigram_counts = compute_ngram_counts(train_sentences, 1)
bigram_counts = compute_ngram_counts(train_sentences, 2)
trigram_counts = compute_ngram_counts(train_sentences, 3)

# Estimate the conditional probabilities for each n-gram model
k = 0.1  # smoothing parameter
vocab_size = len(set(nltk.word_tokenize(data)))
unigram_probs = {ngram: estimate_conditional_prob(ngram, unigram_counts.copy(), k, vocab_size) for ngram in unigram_counts.keys()}
bigram_probs = {ngram: estimate_conditional_prob(ngram, bigram_counts.copy(), k, vocab_size) for ngram in bigram_counts.keys()}
trigram_probs = {ngram: estimate_conditional_prob(ngram, trigram_counts.copy(), k, vocab_size) for ngram in trigram_counts.keys()}


This code first defines two functions: `compute_ngram_counts` and `estimate_conditional_prob`. The `compute_ngram_counts` function takes a list of sentences and an integer `n` as input, and returns a dictionary with the counts of all `n`-grams in the input sentences. The `estimate_conditional_prob` function takes an `n`-gram, a dictionary with `n`-gram counts, a smoothing parameter `k`, and the vocabulary size as input, and returns the estimated conditional probability of the next word in the `n`-gram using `k`-smoothing.

The code then computes the count of unigrams, bigrams, and trigrams from the training data using the `compute_ngram_counts` function, and estimates the conditional probabilities for each `n`-gram model using the `estimate_conditional_prob` function. The smoothing parameter `k` and the vocabulary size are set to 0.1 and the number of unique words in the data, respectively. The estimated conditional probabilities are stored in dictionaries `unigram_probs`, `bigram_probs`, and `trigram_probs`

## 3.Evaluate the N-gram models by computing the perplexity score.
To evaluate the N-gram models, we can compute the perplexity score for each model on the test data. Perplexity is a measure of how well a probability model predicts a sample. It is defined as the inverse of the probability of the test set, normalized by the number of words in the test set. A lower perplexity score indicates a better probability model.

In [28]:
import math

def compute_perplexity(sentences, ngram_probs, n):
    # Compute the perplexity score for a given set of sentences and n-gram probabilities
    perplexity = 0
    num_words = 0
    for sentence in sentences:
        prob = 1
        for i in range(len(sentence) - n + 1):
            ngram = tuple(sentence[i:i+n])
            prob *= ngram_probs.get(ngram, 0)
            num_words += 1
        if prob > 0:  # add a check to ensure that the probability is not zero
            perplexity += math.log(prob)
    perplexity = math.exp(-perplexity / num_words)
    return perplexity

# Compute the perplexity scores for each n-gram model
unigram_perplexity = compute_perplexity(test_sentences, unigram_probs, 1)
bigram_perplexity = compute_perplexity(test_sentences, bigram_probs, 2)
trigram_perplexity = compute_perplexity(test_sentences, trigram_probs, 3)

print("Unigram perplexity:", unigram_perplexity)
print("Bigram perplexity:", bigram_perplexity)
print("Trigram perplexity:", trigram_perplexity)


Unigram perplexity: 2.4393821972444636
Bigram perplexity: 1.161429888271076
Trigram perplexity: 1.0352450178204275


This code defines a function `compute_perplexity` that takes a list of sentences, a dictionary with n-gram probabilities, and an integer `n` as input, and returns the perplexity score for the input sentences and n-gram probabilities. The function first initializes the perplexity score and the number of words to 0, and then iterates over the sentences in the input list. For each sentence, it computes the probability of the sentence using the input n-gram probabilities, and adds the log of the probability to the perplexity score. Finally, the perplexity score is normalized by the number of words and exponentiated to get the final perplexity score.

The code then calls the `compute_perplexity` function on the test data and the n-gram probabilities for each model, and prints the perplexity scores for the unigram, bigram, and trigram models.

## 4. Use your own model to suggest an upcoming word given your sentence.
To suggest an upcoming word given a sentence using own N-gram model, we can use the estimated conditional probabilities of the N-grams in the model to predict the most likely next word.

In [31]:
def suggest_next_word(sentence, ngram_probs, n, vocab):
    # Suggest the next word in a sentence using n-gram probabilities
    sentence = sentence.lower().strip()
    sentence = nltk.word_tokenize(sentence)
    sentence = [word if word_freq[word] > unk_threshold else '<unk>' for word in sentence]
    sentence = tuple(sentence[-n+1:])  # get the last n-1 words of the sentence as the context
    next_word = ''
    max_prob = 0
    for word in vocab:
        ngram = sentence + (word,)
        prob = ngram_probs.get(ngram, 0)
        if prob > max_prob:
            max_prob = prob
            next_word = word
    return next_word

In [33]:
# Test the suggest_next_word function
vocab = set(nltk.word_tokenize(data))
sentence = 'human nature is exceedingly'
next_word = suggest_next_word(sentence, trigram_probs, 3, vocab)
print('Suggested next word:', next_word)  # should print 'mat'

Suggested next word: stubborn


This code defines a function `suggest_next_word` that takes a sentence, a dictionary with n-gram probabilities, and an integer `n` as input, and returns the most likely next word in the sentence according to the input n-gram probabilities. The function first preprocesses the input sentence by lowercasing it, stripping leading and trailing white space, and replacing low-frequency words with the unknown marker `<unk>`. It then extracts the last `n-1` words of the sentence as the context for predicting the next word, and iterates over the vocabulary to find the word with the highest probability given the context. The function returns the word with the highest probability as the suggested next word.

The code then calls the `suggest_next_word` function on a test sentence and the trigram probabilities, and prints the suggested next word. In this example, the suggested next word should be 'mat'. Note that the performance of the suggest_next_word function will depend on the quality of the N-gram model and the smoothing parameter `k` used to estimate the conditional probabilities.